<a href="https://colab.research.google.com/github/Gustavo-br-rj/PROJ-BI_MASTER/blob/main/3_SALVA_MODELOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Previsão de Série Temporal da Temperatura de Mancais Utilizando LSTM

## Parte 0 - Importação das Bibliotecas


In [2]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import json

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#tf.__version__

## Parte 1 - Importação do Dataset

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/DATASETS/dataset_C-4451_08001A.csv')

#Conversão do campo data para datetime e ordenação do dataset
dataset['DATA'] = pd.to_datetime(dataset['DATA'],dayfirst=True)
dataset = dataset.sort_values(['EQUIPAMENTO', 'MANCAL', 'DATA'], ascending=[1, 1, 1])

## Parte 2 - Declaração de Funções

In [4]:
#Função para conversão para tipos serializáveis 
def converter(obj):
  if isinstance(obj, np.integer):
    return int(obj)
  elif isinstance(obj, np.floating):
    return float(obj)
  elif isinstance(obj, np.ndarray):
    return obj.tolist()
  elif isinstance(obj, datetime.datetime):
    return obj.__str__()

In [5]:
# Função para retornar os melhores parâmetros para a construção da RN do mancal, considerando a métrica e o output
def returnBestParams(data_set_param, metric, output):
  #RMSE, MSE, MAPE
  if ( metric == 'RMSE' or metric == 'MSE' or metric == 'MAPE' ):
    ds_aux = data_set_param.loc[data_set_param['OUTPUT'] == output]
    bestParams = ds_aux.loc[ds_aux[metric] == ds_aux[metric].min()]

  return bestParams

In [6]:
# Função para criar a estrutura de dados com janela WINDOW e output OUTPUTSIZE para entrada na RN
def prepareSetRN(windowSize, outputSize, flatSet ):
  preparedX = []
  preparedY = []
  setSize = len(flatSet)
  for i in range(windowSize, setSize - (outputSize-1)):
    preparedX.append(flatSet[i-windowSize:i, 0])
    preparedY.append(flatSet[i:i + outputSize, 0])

  preparedX, preparedY = np.array(preparedX), np.array(preparedY)
  return preparedX, preparedY

In [7]:
# Função para criar a RN com base nos parâmetros informados
def criaRN(outputParam, layerParam, unitParam, dropoutParam, optmizerParam, X_trainParam):
  # Inicializar a RNN
  regressor = Sequential()

  # Adicionar a primeira camada LSTM e Dropout 
  regressor.add(LSTM(units = unitParam[0], return_sequences = (layerParam > 1), input_shape = (X_trainParam.shape[1], 1)))
  regressor.add(Dropout(dropoutParam))
 
  if ( layerParam >= 2 ):
    # Adicionar a segunda camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[1], return_sequences = (layerParam > 2)))
    regressor.add(Dropout(dropoutParam))

  if ( layerParam >= 3):
    # Adicionar a terceira camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[2], return_sequences = (layerParam > 3)))
    regressor.add(Dropout(dropoutParam))

  # camada de saída
  regressor.add(Dense(units = outputParam, activation='linear'))
  #Testar sigmoid e linear

  # Compilar a rede
  regressor.compile(optimizer = optmizerParam, loss = 'mean_squared_error', metrics=['accuracy'])

  # Visualizar a rede
  #regressor.summary()

  return regressor

# 3 - Obtendo os melhores parâmetros encontrados para a criação da RN de cada Mancal 

In [ ]:
# Importação dos resultados da avaliação dos parâmetros da RN para obtenção do melhor resultado para cada MANCAL
path = '/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/AVALIACAO_REDES/'

files = os.listdir(path)

for f in files:
  if f.endswith("_results.csv"):
    result4RN = pd.read_csv(path + f)
    bestParams = returnBestParams(result4RN, 'RMSE', 3 )

    equipamento = bestParams.iloc[0]['EQUIPAMENTO']
    mancal      = bestParams.iloc[0]['MANCAL']
    window      = bestParams.iloc[0]['WINDOW']
    output      = bestParams.iloc[0]['OUTPUT']
    layer       = bestParams.iloc[0]['NUM. CAMADAS']
    unit1       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 1']
    unit2       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 2']
    unit3       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 3']
    unit        = [unit1,unit2,unit3]
    dropout     = bestParams.iloc[0]['DROPOUT']
    optmizer    = bestParams.iloc[0]['OTIMIZADOR']
    epochs      = bestParams.iloc[0]['EPOCHS']
    batch_size  = bestParams.iloc[0]['BATCH_SIZE']

    datasetMancal = dataset.loc[(dataset['MANCAL'] == mancal)]
    #Calculando tamanho dos conjuntos de treino e teste
    train_size = math.ceil(len(datasetMancal)*0.85)
    test_size  = len(datasetMancal) - train_size

    #Separando conjuto de treino e teste apenas com valor da série
    train_set = datasetMancal.iloc[0:train_size, 3:4].values
    test_set  = datasetMancal.iloc[train_size:(train_size + test_size), 3:4].values

    #Normalização
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(train_set)

    X_train = []
    y_train = []
    
    X_train, y_train = prepareSetRN(window, output, training_set_scaled)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    regressor = criaRN(output, layer, unit, dropout, optmizer, X_train)
    regressor.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

    #Salvar modelo
    dirModels = '/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/'
    nomeModelo = '{0}{1}__{2}_model'.format(dirModels,equipamento, mancal)
    regressor.save(nomeModelo)

    #Salvar parâmetros
    nomeArq   = '{0}__{1}_params.json'.format(equipamento, mancal)
    arquivo   = '{0}{1}'.format(dirModels,nomeArq)
    fParams   = open(arquivo,"w+")

    param_set = {"equipamento": equipamento, "mancal": mancal, "window": window, 
                 "output": output, "layer": layer, "unit": unit, "dropout": dropout, 
                 "optmizer": optmizer, "epochs": epochs, "batch_size": batch_size}

    json_dump = json.dumps(param_set, default=converter)
    fParams.write(json_dump)
    fParams.close()  